# Decorator vs Class Approach
The goal is to profile a function by keeping track of :
- how many times it has been run
- its total run time

We'll see two approaches. The first one uses decorator. It is a functionnal approach, that is a bit cumbersome here but is in general much simpler. The second one is object oriented. It is more elegant in this case but might be a bit of an overkill for simple cases.

## A. Decorator approach 

In [3]:
from time import perf_counter, sleep
from random import random

#### a. broken example
Below we use an **immutable** object to keep track of the count. It won't work as expected because of the way we have implemented it. See comments in the function.

In [5]:
def profiler(func):

    # _counter points to object 0
    _counter = 0 # immutable object

    def inner(*args, **kwargs):
        
        nonlocal _counter

        # _counter points to different objects at each call, i.e 1, 2, 3, ...
        # taht is because int are immutable object
        
        _counter += 1
        result = func(*args, **kwargs)
        
        return result

    # counter points to object 0
    inner.counter = _counter 
    return inner


@profiler
def function():
    sleep(random())
    return True    

In [6]:
function(), function(), function()

(True, True, True)

In [7]:
function.counter

0

Answer above is incorrect because ```_counter``` and ```count``` points toward different objects.

We can fix it by using mutable object instead.

In [9]:
def profiler(func):

    # _counter points to list
    _counter = [0] # mutable object

    def inner(*args, **kwargs):
        
        nonlocal _counter
        _counter[0] += 1
        result = func(*args, **kwargs)
        
        return result

    inner.counter = _counter 
    return inner


@profiler
def function():
    sleep(random())
    return True 

In [10]:
function(), function(), function()

(True, True, True)

In [11]:
function.counter

[3]

Here ```_counter``` and ```count``` points to the same list object that is why answer is correct.

If we want to use immutable object, we must change our implementation.

In [14]:
def profiler(func):

    _counter = 0

    def inner(*args, **kwargs):
        
        nonlocal _counter
        _counter += 1
        result = func(*args, **kwargs)
        
        return result

    # now counter() points to the same object as _counter
    def counter():
        return _counter 
        
    inner.counter = counter
    return inner


@profiler
def function():
    sleep(random())
    return True 

In [15]:
function(), function(), function()

(True, True, True)

In [16]:
function.counter()

3

#### b. complete example

In [18]:
def profiler(func):

    _counter = 0
    _elapsed_time = 0

    def inner(*args, **kwargs):
        
        nonlocal _counter
        nonlocal _elapsed_time
        
        _counter += 1
    
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()

        _elapsed_time += end-start

        
        return result
        
    def counter():
        return _counter

    def elapsed_time():
        return _elapsed_time

    def avg_elapsed_time():
        return _elapsed_time/_counter

    inner.counter = counter
    inner.elapsed_time = elapsed_time
    inner.avg_elapsed_time = avg_elapsed_time
    return inner


@profiler
def function():
    sleep(random())
    return True 

In [19]:
function(), function(), function()

(True, True, True)

In [20]:
function.counter(), function.elapsed_time(), function.avg_elapsed_time()

(3, 1.8992454998660833, 0.6330818332886944)

## B. Class approach

In [22]:
class Profiler:

    def __init__(self, func):
        self.func = func
        self.counter = 0
        self.elapsed_time = 0

    def __call__(self, *args, **kwargs):
        self.counter += 1
        start = perf_counter()
        result = self.func(*args, **kwargs)
        end = perf_counter()
        self.elapsed_time += end-start
        return result

    @property
    def avg_elapsed_time(self):
        return self.elapsed_time/self.counter

def function():
    sleep(random())
    return True 

In [23]:
profiled_func = Profiler(function) # create a Profiler instance

In [24]:
profiled_func(), profiled_func(), profiled_func() 

(True, True, True)

In [25]:
profiled_func.counter, profiled_func.elapsed_time, profiled_func.avg_elapsed_time

(3, 1.7510794999543577, 0.5836931666514525)

We can also make use of the syntactic sugar expression

In [27]:
@Profiler
def function():
    sleep(random())
    return True 

In [28]:
function(), function(), function()

(True, True, True)

In [29]:
function.counter, function.elapsed_time, function.avg_elapsed_time

(3, 0.959229200379923, 0.31974306679330766)